# Accessibility Report
## The result is a csv with the candidate, position, website, vendor along with payment and the fields from their campaign website's axe report on accessibility.

In [1]:
import json
import pandas as pd
import csv
from selenium import webdriver
from axe_selenium_python import Axe
import requests
from requests.exceptions import ConnectionError
import os

In [2]:
candidate_data = pd.read_csv(r'Seattle-2021ElectionWebsiteData - Sheet1.csv')

### The following are all the available features in this dataset:

In [3]:
list(candidate_data.columns)

['Position', 'Candidate', 'Website ', 'Vendor', 'Payment ', 'Source']

### The following features will be included in the final dataset:
- Postion
- Candidate
- Website
- Vendor
- Payment

### The following features will be included from the axe report:
- Violation #
- Description
- Help
- HelpUrl
- Id
- Impact
- Tags
- Website


This function accepts a url and creates an axe report in json format. Then parses the json file to create a dictionary with all the elements nested.

In [4]:
def test_site(url: str) -> dict:
    # Add the path the geckodriver you downloaded earlier
    # The following is an example
    driver = webdriver.Firefox()
    driver.get(url)
    axe = Axe(driver)
    # Inject axe-core javascript into page.
    axe.inject()
    # Run axe accessibility checks.
    results = axe.run()
    # Write results to file
    axe.write_results(results, 'access.json')
    driver.close()
    # Assert no violations are found
    # assert len(results["violations"]) == 0, axe.report(results["violations"])
    with open('access.json', 'r') as content:
        data = json.load(content)
    
    return data

In [5]:
# Testing the function:

test_data = test_site("http://natalieisawesome.com/")

In [6]:
result = pd.DataFrame(columns = ["Position", "Candidate",
                                 "Violation #", "Description", "Help", "HelpUrl", "Id", "Impact", "Tags"])

In [7]:
for index, row in candidate_data.iterrows():
    from requests.exceptions import MissingSchema
    website = row['Website ']
    if type(row['Website ']) == str:  
        if ".org" in website or ".com" in website:
            if "http" not in website:
                website = ''.join(('https://', website))
                
    # checking if there is a value for 'website'
    if not pd.isna(website):
        try:
            # checking if this website is valid
            request = requests.get(website)
        except ConnectionError:
            print(row['Candidate']+ '\'s campaign website does not exist')
            continue
        except MissingSchema:
            print(row['Candidate']+ '\'s website url not present')
            continue
        
        # calling the function to create json report file
        axe_report = test_site(website)
        axe_violations = axe_report['violations']
        count = len(axe_violations)
        print("Checking violations in " + row['Candidate']+ '\'s campaign website')
        
        # appending the data frame
        # each violation is added as a new observation and tracked with vilation #
        for i in range(count):
            violation_num = i + 1
            violation_desc = axe_violations[i]['description']
            violation_help = axe_violations[i]['help']
            violation_helpurl = axe_violations[i]['helpUrl']
            violation_id = axe_violations[i]['id']
            violation_impact = axe_violations[i]['impact']
            violation_tags = axe_violations[i]['tags']
            violation_nodes = axe_violations[i]['nodes']
            node_count = len(violation_nodes)
            violation_html_list = {}
            for j in range(node_count):
                failureSummary = violation_nodes[j]['failureSummary']
                html = violation_nodes[j]['html']
                violation_html_list[failureSummary] = html
            result = result.append({'Position': row['Position'], 
                                    'Candidate': row['Candidate'], 
                                    'Website ': row['Website '], 'Violation #': violation_num,'failureSummary:html': violation_html_list,
                                    'Description': violation_desc, 'Help': violation_help, 'HelpUrl': violation_helpurl,
                                    'Id': violation_id, 'Impact': violation_impact, 'Tags': violation_tags}, ignore_index=True)

Checking violations in Bliss's campaign website
Checking violations in Donaldson's campaign website
Checking violations in Echohawk's campaign website
Checking violations in Farrell's campaign website
Checking violations in Gonzalez's campaign website
Checking violations in Harrell's campaign website
Checking violations in Houston's campaign website
Lippmann's website url not present
Checking violations in Randall's campaign website
Checking violations in Rivers's campaign website
Checking violations in Sixkiller's campaign website
Checking violations in Tucker's campaign website
Checking violations in Holmes's campaign website
Checking violations in Thomas-Kennedy's campaign website
Checking violations in Glumaz's campaign website
Checking violations in Martin's campaign website
Checking violations in Mosqueda's campaign website
Checking violations in Tsimerman's campaign website
Checking violations in Echner's campaign website
Checking violations in Gunther's campaign website
Checkin

In [8]:
result.head()

,Position,Candidate,Violation #,Description,Help,HelpUrl,Id,Impact,Tags,Website,failureSummary:html
0,MAYOR,Bliss,1,Ensures role attribute has an appropriate valu...,ARIA role must be appropriate for the element,https://dequeuniversity.com/rules/axe/3.1/aria...,aria-allowed-role,minor,"[cat.aria, best-practice]",www.doctorblissformayor.com,{'Fix any of the following: role button is ...
1,MAYOR,Bliss,2,Ensures the contrast between foreground and ba...,Elements must have sufficient color contrast,https://dequeuniversity.com/rules/axe/3.1/colo...,color-contrast,serious,"[cat.color, wcag2aa, wcag143]",www.doctorblissformayor.com,{'Fix any of the following: Element has insu...
2,MAYOR,Bliss,3,Ensures every id attribute value is unique,id attribute value must be unique,https://dequeuniversity.com/rules/axe/3.1/dupl...,duplicate-id,minor,"[cat.parsing, wcag2a, wcag411]",www.doctorblissformayor.com,{'Fix any of the following: Document has mul...
3,MAYOR,Bliss,4,Ensures <iframe> and <frame> elements contain ...,Frames must have title attribute,https://dequeuniversity.com/rules/axe/3.1/fram...,frame-title,serious,"[cat.text-alternatives, wcag2a, wcag241, wcag4...",www.doctorblissformayor.com,{'Fix any of the following: aria-label attri...
4,MAYOR,Bliss,5,Ensures every form element has a label,Form elements must have labels,https://dequeuniversity.com/rules/axe/3.1/labe...,label,critical,"[cat.forms, wcag2a, wcag332, wcag131, section5...",www.doctorblissformayor.com,{'Fix all of the following: Form element has...


Converting the dataframe to CSV:

In [9]:
result.to_csv('new_seattle_2021_candidate_website_violations.csv', encoding='utf-8')